# **DL Challange 2018**

Drive mount

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


## Data Copy

In [2]:
# %rm data.tar.xz
# %rm -rf dataAll
%ls /gdrive/'My Drive'/ML/Hackerearth/Animal-classifier
%cp /gdrive/'My Drive'/ML/Hackerearth/Animal-classifier/data.zip /content/

alien_test/  data.zip  DLBeginnerorganized.zip  DLBeginner.zip  meta-data.zip


In [0]:
import zipfile
with zipfile.ZipFile("data.zip", 'r') as zip_ref:
    zip_ref.extractall("")

In [14]:
%ls data
%rm train_data.dat
%ls

0/  10/  12/  14/  16/  18/  2/   21/  23/  25/  27/  3/  5/  7/  9/
1/  11/  13/  15/  17/  19/  20/  22/  24/  26/  28/  4/  6/  8/
alien_test/  data/  data.zip  sample_data/


## **Imports**

In [5]:
import os
import cv2
import numpy as np
from tqdm import tqdm
from random import shuffle
import pickle

import keras
from keras.utils import to_categorical
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

Using TensorFlow backend.


### Global Variables

In [0]:
IMG_SIZE = 150
LR = 1e-3
batch_size = 16
num_classes = 29
epochs = 5
data_augmentation = False
num_predictions = 20
model_name = 'trained_model.h5'

# Data Preprocessing



In [16]:
#%rm train_data.dat
path = "data"

def create_train_data():
  training_data = []

  if os.path.exists("train_data.dat"):
    file = open('train_data.dat', 'rb')
    training_data = pickle.load(file)
    file.close()
    return training_data

  #img_count = 0
  for folder in tqdm(os.listdir(path)):
    p = path + "/" + folder
    files = os.listdir(p)
    for i in files:
      label = folder
      img_loc = p + "/" + i
      img = cv2.imread(img_loc,cv2.IMREAD_GRAYSCALE)
      img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
      training_data.append([np.array(img),np.array(label)])
      #img_count += 1
      #if img_count > 6000 : break

  shuffle(training_data)

  file = open('train_data.dat', 'wb')
  pickle.dump(training_data, file)  
  file.close()

  return training_data

train_data = create_train_data()
%ls

100%|██████████| 29/29 [00:04<00:00,  6.68it/s]


alien_test/  data/  data.zip  sample_data/  train_data.dat


In [17]:
%cp /content/train_data.dat /gdrive/'My Drive'/ML/Hackerearth/Animal-classifier/
%ls /gdrive/'My Drive'/ML/Hackerearth/Animal-classifier/

alien_test/  DLBeginnerorganized.zip  meta-data.zip
data.zip     DLBeginner.zip           train_data.dat


## Data Split

In [18]:
train = train_data[:-3000]
test = train_data[-3000:]
# Training Data
x_train = np.array([i[0] for i in train]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
y_train = [i[1] for i in train]

y_train = to_categorical(y_train,29)
print(y_train)

# Testing Data
x_test = np.array([i[0] for i in test]).reshape(-1,IMG_SIZE,IMG_SIZE,1)

y_test = [i[1] for i in test]
print(y_test)
y_test = to_categorical(y_test,29)
print(y_test)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[array('28', dtype='<U2'), array('3', dtype='<U1'), array('12', dtype='<U2'), array('27', dtype='<U2'), array('7', dtype='<U1'), array('6', dtype='<U1'), array('1', dtype='<U1'), array('17', dtype='<U2'), array('10', dtype='<U2'), array('9', dtype='<U1'), array('9', dtype='<U1'), array('0', dtype='<U1'), array('0', dtype='<U1'), array('10', dtype='<U2'), array('7', dtype='<U1'), array('5', dtype='<U1'), array('21', dtype='<U2'), array('5', dtype='<U1'), array('21', dtype='<U2'), array('2', dtype='<U1'), array('22', dtype='<U2'), array('22', dtype='<U2'), array('7', dtype='<U1'), array('21', dtype='<U2'), array('17', dtype='<U2'), array('13', dtype='<U2'), array('16', dtype='<U2'), array('0', dtype='<U1'), array('22', dtype='<U2'), array('25', dtype='<U2'), array('16', dtype='<U2'), array('6', dtype='<U1'), array('12', dtype='<U2'), 

## Design Model

In [0]:
model = Sequential()
model.add(Conv2D(64, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# Initiate RMSprop optimizer
#opt = keras.optimizers.rmsprop(lr=0.001, decay=1e-6)
opt = keras.optimizers.Adam(0.0005, beta_1=0.9, beta_2=0.999, amsgrad=True)
#opt = keras.optimizers.SGD(lr=0.01, momentum=0.0, nesterov=False)

# Model Train

In [20]:
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)

Not using data augmentation.
Train on 10000 samples, validate on 3000 samples
Epoch 1/5
10000/10000 [==============================] - 112s 11ms/step - loss: 3.2698 - accuracy: 0.0784 - val_loss: 3.1987 - val_accuracy: 0.0930
Epoch 2/5
10000/10000 [==============================] - 111s 11ms/step - loss: 3.1534 - accuracy: 0.1104 - val_loss: 3.0395 - val_accuracy: 0.1467
Epoch 3/5
10000/10000 [==============================] - 111s 11ms/step - loss: 2.9839 - accuracy: 0.1553 - val_loss: 2.9229 - val_accuracy: 0.1797
Epoch 4/5
10000/10000 [==============================] - 111s 11ms/step - loss: 2.6256 - accuracy: 0.2471 - val_loss: 2.8042 - val_accuracy: 0.2087
Epoch 5/5
10000/10000 [==============================] - 111s 11ms/step - loss: 1.9038 - accuracy: 0.4487 - val_loss: 2.9074 - val_accuracy: 0.2253


## Results

In [21]:
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

3000/3000 [==============================] - 6s 2ms/step
Test loss: 2.9073532123565675
Test accuracy: 0.2253333330154419


## Saving the model

In [22]:
# Save model and weights
model.save(model_name)
%cp /content/trained_model.h5 /gdrive/'My Drive'/ML/Hackerearth/Animal-classifier/
%ls /gdrive/'My Drive'/ML/Hackerearth/Animal-classifier/

alien_test/  DLBeginnerorganized.zip  meta-data.zip   trained_model.h5
data.zip     DLBeginner.zip           train_data.dat


# Model Check

## Data Intake

In [0]:
def create_alientest_data():
    alien_test = []
    for i in os.listdir(test_path):
        img_loc = test_path+ "/" + i
        print(img_loc)
        try:
            img = cv2.imread(img_loc,cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
            alien_test.append([np.array(img),np.array("NULL")])
        except:
            print("Error at :"+ img_loc)
    return alien_test

## Labelling

In [0]:
def labelling(result):
  #print(result)
  for i in range(result.shape[0]):
    #print(i)
    answer = 0
    for j in range(result[i].shape[0]):
      if(result[i][j]==1):
        #print(answer)
        if answer == 0:
          print("antelope")
        elif answer == 1:
          print("bat")
        elif answer == 2:
          print("beaver")
        elif answer == 3:
          print("bobcat")
        elif answer == 4:
          print("buffalo")
        elif answer == 5:
          print("chihuahua")
        elif answer == 6:
          print("chimpanzee")
        elif answer == 7:
          print("collie")
        elif answer == 8:
          print("dalmatian")
        elif answer == 9:
          print("germanshepherd")
        elif answer == 10:
          print("grizzlybear")
        elif answer == 11:
          print("hippopotamus")
        elif answer == 12:
          print("horse")
        elif answer == 13:
          print("killerwhale")
        elif answer == 14:
          print("mole")
        elif answer == 15:
          print("mouse")
        elif answer == 16:
          print("otter")
        elif answer == 17:
          print("ox")
        elif answer == 18:
          print("persiancat")
        elif answer == 19:
          print("raccoon")
        elif answer == 20:
          print("rat")
        elif answer == 21:
          print("rhinoceros")
        elif answer == 22:
          print("seal")
        elif answer == 23:
          print("siamesecat")
        elif answer == 24:
          print("spidermonkey")
        elif answer == 25:
          print("squirrel")
        elif answer == 26:
          print("walrus")
        elif answer == 27:
          print("weasel")
        elif answer == 28:
          print("wolf")
        else:
          print("Other")
      answer += 1

## Model Setup


In [0]:
from keras.models import Sequential, load_model

IMG_SIZE = 150

model_path = 'trained_model.h5'
model = load_model(model_path)
test_path = '/gdrive/My Drive/ML/Hackerearth/Animal-classifier/alien_test'

#opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)
opt = keras.optimizers.Adam(0.0005, beta_1=0.9, beta_2=0.999, amsgrad=True)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

## Model.Predict

In [26]:
train = create_alientest_data()
x_train = np.array([i[0] for i in train]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
array = model.predict(x_train)
labelling(array)

/gdrive/My Drive/ML/Hackerearth/Animal-classifier/alien_test/Img-4.jpg
/gdrive/My Drive/ML/Hackerearth/Animal-classifier/alien_test/Img-13.jpg
/gdrive/My Drive/ML/Hackerearth/Animal-classifier/alien_test/Img-41.jpg
/gdrive/My Drive/ML/Hackerearth/Animal-classifier/alien_test/Img-17.jpg
/gdrive/My Drive/ML/Hackerearth/Animal-classifier/alien_test/Img-8.jpg
/gdrive/My Drive/ML/Hackerearth/Animal-classifier/alien_test/Img-20.jpg
/gdrive/My Drive/ML/Hackerearth/Animal-classifier/alien_test/Img-47.jpg
/gdrive/My Drive/ML/Hackerearth/Animal-classifier/alien_test/Img-22.jpg
/gdrive/My Drive/ML/Hackerearth/Animal-classifier/alien_test/Img-44.jpg
/gdrive/My Drive/ML/Hackerearth/Animal-classifier/alien_test/Img-23.jpg
/gdrive/My Drive/ML/Hackerearth/Animal-classifier/alien_test/Img-27.jpg
/gdrive/My Drive/ML/Hackerearth/Animal-classifier/alien_test/Img-45.jpg
/gdrive/My Drive/ML/Hackerearth/Animal-classifier/alien_test/Img-46.jpg
/gdrive/My Drive/ML/Hackerearth/Animal-classifier/alien_test/Img-3